#forest_fire_detection

In [ ]:
import cv2
import os
import numpy as np


In [ ]:
!pip install matplotlib


#import necessary libraries

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


#read the zip file

In [ ]:
import zipfile
import io
import cv2
import numpy as np

# Specify the path to the uploaded ZIP archive
zip_path = '/content/archive (2).zip'  # Update with your ZIP file path

# Create a dictionary to store image data and labels
data = {
    'train': {'images': [], 'labels': []},
    'validation': {'images': [], 'labels': []}
}

# Open the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    for set_type in ['train', 'validation']:
        set_path = f'{set_type}/{set_type}/'

        for class_name in ['fire', 'non_fire']:
            class_path = set_path + class_name
            label = 1 if class_name == 'fire' else 0  # Assign labels (1 for fire, 0 for non-fire)

            for image_name in zip_ref.namelist():
                if image_name.startswith(class_path):
                    with zip_ref.open(image_name) as file:
                        image_data = io.BytesIO(file.read())
                        image = cv2.imdecode(np.frombuffer(image_data.read(), np.uint8), cv2.IMREAD_COLOR)
                        image = cv2.resize(image, (128, 128))  # Adjust size as needed
                        data[set_type]['images'].append(image)
                        data[set_type]['labels'].append(label)

# Convert data to NumPy arrays
train_images = np.array(data['train']['images']) / 255.0
train_labels = np.array(data['train']['labels'])
validation_images = np.array(data['validation']['images']) / 255.0
validation_labels = np.array(data['validation']['labels'])


#create a extraction file

In [ ]:
zip_path = '/content/archive (2).zip'
extraction_path = 'path/to/your/extraction_folder'


In [ ]:
if not os.path.exists(extraction_path):
    os.makedirs(extraction_path)


In [ ]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

# List the folders within the extraction directory
extracted_folders = [folder for folder in os.listdir(extraction_path) if os.path.isdir(os.path.join(extraction_path, folder))]

# Print the list of extracted folders
for folder in extracted_folders:
    print(f'Extracted folder: {folder}')


Extracted folder: forest_fire


In [ ]:
import zipfile
import os

# Specify the path to the uploaded ZIP archive
zip_path = '/content/archive (2).zip'

# Specify the folder where you want to extract the contents
extraction_path = '/content/extracted_folder'

# Create the extraction folder if it doesn't exist
if not os.path.exists(extraction_path):
    os.makedirs(extraction_path)

try:
    # Unzip the archive
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extraction_path)
    print(f'Successfully extracted files to {extraction_path}')
except zipfile.BadZipFile:
    print(f'The provided file is not a valid ZIP archive.')
except Exception as e:
    print(f'An error occurred: {str(e)}')


Successfully extracted files to /content/extracted_folder


In [ ]:
import os

# Specify the path to the extracted folder
extracted_folder = '/content/extracted_folder'

# List all files and subdirectories in the extracted folder
contents = os.listdir(extracted_folder)

# Print the contents
for item in contents:
    item_path = os.path.join(extracted_folder, item)
    if os.path.isfile(item_path):
        print(f"File: {item}")
    elif os.path.isdir(item_path):
        print(f"Directory: {item}")


Directory: forest_fire


#read the extracted file

In [ ]:
import os

# Specify the path to the folder you want to inspect
folder_path = '/content/extracted_folder/forest_fire'  # Update with the actual path

# List all files and subdirectories in the folder
contents = os.listdir(folder_path)

# Print the contents
for item in contents:
    item_path = os.path.join(folder_path, item)
    if os.path.isfile(item_path):
        print(f"File: {item}")
    elif os.path.isdir(item_path):
        print(f"Directory: {item}")


Directory: Testing
Directory: Training and Validation


In [ ]:
import os

# Specify the path to the main folder containing the subfolders
main_folder = '/content/extracted_folder/forest_fire'  # Update with the actual path

# Function to list and preview details of files in a folder
def preview_files_in_folder(folder_path):
    contents = os.listdir(folder_path)
    for item in contents:
        item_path = os.path.join(folder_path, item)
        if os.path.isfile(item_path):
            # Print file details
            print(f"File: {item}")
            print(f"File size (bytes): {os.path.getsize(item_path)}")
            # Add more details if needed
            print()
        elif os.path.isdir(item_path):
            # Recursively preview files in subfolders
            preview_files_in_folder(item_path)

# Call the function to preview files in both subfolders
preview_files_in_folder(main_folder)


Streaming output truncated to the last 5000 lines.
File size (bytes): 18123

File: nofire_0670.jpg
File size (bytes): 19357

File: abc286.jpg
File size (bytes): 11998

File: nofire_0699.jpg
File size (bytes): 15248

File: nofire_0589.jpg
File size (bytes): 13904

File: nofire_0549.jpg
File size (bytes): 11965

File: abc262.jpg
File size (bytes): 12032

File: nofire_0499.jpg
File size (bytes): 14539

File: nofire_0655.jpg
File size (bytes): 13784

File: nofire_0059.jpg
File size (bytes): 20842

File: nofire_0065.jpg
File size (bytes): 15526

File: nofire_0228.jpg
File size (bytes): 19241

File: nofire_0600.jpg
File size (bytes): 12430

File: abc247.jpg
File size (bytes): 21663

File: nofire_0118.jpg
File size (bytes): 26111

File: nofire_0145.jpg
File size (bytes): 16024

File: abc213.jpg
File size (bytes): 24362

File: abc254.jpg
File size (bytes): 11346

File: abc212.jpg
File size (bytes): 19904

File: abc234.jpg
File size (bytes): 28380

File: nofire_0023.jpg
File size (bytes): 19280

###preprocessing and CNN application

#training

In [ ]:
# Specify the paths to your image directories
non_fire_path = '/content/extracted_folder/forest_fire/Training and Validation/nofire'
fire_path = '/content/extracted_folder/forest_fire/Training and Validation/fire'

# Initialize lists to store images and labels
data = []
labels = []

# Load non-fire images
for image_file in os.listdir(non_fire_path):
    if image_file.endswith('.jpg'):
        image_path = os.path.join(non_fire_path, image_file)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (128, 128))  # Resize as needed
        data.append(image)
        labels.append(0)  # Label 0 for non-fire

# Load fire images
for image_file in os.listdir(fire_path):
    if image_file.endswith('.jpg'):
        image_path = os.path.join(fire_path, image_file)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (128, 128))  # Resize as needed
        data.append(image)
        labels.append(1)  # Label 1 for fire

# Convert data and labels to NumPy arrays
data = np.array(data) / 255.0
labels = np.array(labels)

# Split the data into training and validation sets
train_data, validation_data, train_labels, validation_labels = train_test_split(data, labels, test_size=0.2)


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Define the paths to your image directories
non_fire_path = '/content/extracted_folder/forest_fire/Training and Validation/nofire'
fire_path = '/content/extracted_folder/forest_fire/Training and Validation/fire'

# Initialize lists to store images and labels
data = []
labels = []

# Load non-fire images
for image_file in os.listdir(non_fire_path):
    if image_file.endswith('.jpg'):
        image_path = os.path.join(non_fire_path, image_file)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (128, 128))  # Resize as needed
        data.append(image)
        labels.append(0)  # Label 0 for non-fire

# Load fire images
for image_file in os.listdir(fire_path):
    if image_file.endswith('.jpg'):
        image_path = os.path.join(fire_path, image_file)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (128, 128))  # Resize as needed
        data.append(image)
        labels.append(1)  # Label 1 for fire

# Convert data and labels to NumPy arrays
data = np.array(data) / 255.0
labels = np.array(labels)

# Split the data into training and validation sets
train_data, validation_data, train_labels, validation_labels = train_test_split(data, labels, test_size=0.2)

# Build the CNN model
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_data, train_labels, epochs=10, batch_size=32)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(validation_data, validation_labels)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')


Epoch 1/10
46/46 [==============================] - 44s 921ms/step - loss: 0.4110 - accuracy: 0.8683
Epoch 2/10
46/46 [==============================] - 35s 762ms/step - loss: 0.1569 - accuracy: 0.9454
Epoch 3/10
46/46 [==============================] - 37s 808ms/step - loss: 0.1354 - accuracy: 0.9549
Epoch 4/10
46/46 [==============================] - 37s 814ms/step - loss: 0.1030 - accuracy: 0.9686
Epoch 5/10
46/46 [==============================] - 36s 789ms/step - loss: 0.1028 - accuracy: 0.9625
Epoch 6/10
46/46 [==============================] - 38s 820ms/step - loss: 0.0744 - accuracy: 0.9727
Epoch 7/10
46/46 [==============================] - 46s 977ms/step - loss: 0.0442 - accuracy: 0.9884
Epoch 8/10
46/46 [==============================] - 41s 885ms/step - loss: 0.0587 - accuracy: 0.9802
Epoch 9/10
46/46 [==============================] - 42s 916ms/step - loss: 0.0204 - accuracy: 0.9939
Epoch 10/10
12/12 [==============================] - 2s 185ms/step - loss: 0.2219 - accurac

#testing

In [ ]:
import cv2
import numpy as np

# Load your test image
test_image_path = '/content/ff.jpg'
test_image = cv2.imread(test_image_path)
test_image = cv2.resize(test_image, (128, 128))  # Resize to match the model input size

# Normalize the image
test_image = test_image / 255.0

# Reshape the image to match the input shape expected by the model
test_image = np.expand_dims(test_image, axis=0)

# Make a prediction
prediction = model.predict(test_image)

# The prediction is a probability; you can threshold it to get the class
if prediction[0][0] > 0.5:
    result = 'Fire'
else:
    result = 'Non-Fire'

print(f'The image is classified as: {result}')


1/1 [==============================] - 0s 130ms/step
The image is classified as: Fire


###testing data

In [ ]:
import os
import cv2
import numpy as np

# Path to the parent folder containing "fire" and "non-fire" subfolders
parent_folder = '/content/extracted_folder/forest_fire/Testing'

# Initialize a list to store the results for each image
results = []

# Define a function to preprocess and classify an image
def classify_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (128, 128))  # Resize to match the model input size
    image = image / 255.0  # Normalize
    image = np.expand_dims(image, axis=0)  # Reshape for model input
    prediction = model.predict(image)
    return 'Fire' if prediction[0][0] > 0.5 else 'Non-Fire'

# Iterate through the subfolders and images
for folder_name in os.listdir(parent_folder):
    folder_path = os.path.join(parent_folder, folder_name)
    if os.path.isdir(folder_path):
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            if image_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                result = classify_image(image_path)
                results.append(f'{image_path}: {result}')

# Display the results
for result in results:
    print(result)


1/1 [==============================] - 0s 44ms/step
/content/extracted_folder/forest_fire/Testing/nofire/abc361.jpg: Non-Fire
/content/extracted_folder/forest_fire/Testing/nofire/abc373.jpg: Fire
/content/extracted_folder/forest_fire/Testing/nofire/abc349.jpg: Non-Fire
/content/extracted_folder/forest_fire/Testing/nofire/abc358.jpg: Non-Fire
/content/extracted_folder/forest_fire/Testing/nofire/abc379.jpg: Non-Fire
/content/extracted_folder/forest_fire/Testing/nofire/abc378.jpg: Non-Fire
/content/extracted_folder/forest_fire/Testing/nofire/abc342.jpg: Non-Fire
/content/extracted_folder/forest_fire/Testing/nofire/abc359.jpg: Non-Fire
/content/extracted_folder/forest_fire/Testing/nofire/abc362.jpg: Non-Fire
/content/extracted_folder/forest_fire/Testing/nofire/abc335.jpg: Non-Fire
/content/extracted_folder/forest_fire/Testing/nofire/abc355.jpg: Non-Fire
/content/extracted_folder/forest_fire/Testing/nofire/abc338.jpg: Non-Fire
/content/extracted_folder/forest_fire/Testing/nofire/abc340.jpg: